# 3.2 分类函数

## 3.2.1SGD

随机梯度下降是拟合线性模型的一个简单而高效的方法。主要用于凸损失函数下线性分类器的判别式学习，例如(线性)支持向量机和Logistic回归。在样本量（和特征数）很大时尤为有用。SGD 已成功应用于在文本分类和自然语言处理中经常遇到的大规模和稀疏的机器学习问题。方法 partial_fit 可用于 online learning （在线学习）或基于 out-of-core learning （外存的学习）。<br>

SGDClassifier 和 SGDRegressor 分别用于拟合分类问题和回归问题的线性模型，可使用不同的（凸）损失函数，支持不同的罚项。 例如，设定 loss="log" ，则 SGDClassifier 拟合一个逻辑斯蒂回归模型，而 loss="hinge" 拟合线性支持向量机（SVM）。
Stochastic Gradient Descent （随机梯度下降法）的优势:（1）高效。（2）易于实现 (有大量优化代码的机会)。
Stochastic Gradient Descent （随机梯度下降法）的劣势:（1）SGD 需要一些超参数，例如 regularization （正则化）参数和 number of iterations （迭代次数）。（2）SGD 对 feature scaling （特征缩放）敏感。
 


### 分类

SGDClassifier 类实现了一个简单的随机梯度下降学习例程, 支持不同的 loss functions（损失函数）和 penalties for classification（分类处罚）

![](picture/linear0.png)

作为另一个 classifier （分类器）, 拟合 SGD 我们需要两个 array （数组）：保存训练样本的 size 为 [n_samples, n_features] 的数组 X 以及保存训练样本目标值（类标签）的 size 为 [n_samples] 的数组 Y
```
 from sklearn.linear_model import SGDClassifier
 X = [[0., 0.], [1., 1.]]
 y = [0, 1]
 clf = SGDClassifier(loss="hinge", penalty="l2")
 clf.fit(X, y)
 >> SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False) 
```

拟合后，我们可以用该模型来预测新值:
```
 clf.predict([[2., 2.]])
 >> array([1])
```

SGD 通过训练数据来拟合一个线性模型。成员 coef_ 保存模型参数:
```
 clf.coef_      
 >> array([[ 9.9...,  9.9...]])
```

成员 intercept_ 保存 intercept（截距） （又称作 offset（偏移）或 bias（偏差））:
```
 clf.intercept_     
 >> array([-9.9...])
```

模型是否使用 intercept（截距）, 即 a biased hyperplane(一个偏置的超平面), 是由参数 fit_intercept 控制的。
使用 SGDClassifier.decision_function 来获得到此超平面的 signed distance (符号距离)
```
 clf.decision_function([[2., 2.]])                 
 >> array([ 29.6...])
```

具体的 loss function（损失函数） 可以通过 loss 参数来设置。 SGDClassifier 支持以下的 loss functions（损失函数）：

- loss="hinge": (soft-margin) linear Support Vector Machine （（软-间隔）线性支持向量机），
- loss="modified_huber": smoothed hinge loss （平滑的 hinge 损失），
- loss="log": logistic regression （logistic 回归），
- and all regression losses below（以及所有的回归损失）。

前两个 loss functions（损失函数）是懒惰的，如果一个例子违反了 margin constraint（边界约束），它们仅更新模型的参数, 这使得训练非常有效率,即使使用了 L2 penalty（惩罚）我们仍然可能得到稀疏的模型结果。

使用 loss="log" 或者 loss="modified_huber" 来启用 predict_proba 方法, 其给出每个样本 x 的概率估计 P(y|x) 的一个向量：
```
 clf = SGDClassifier(loss="log").fit(X, y)
 clf.predict_proba([[1., 1.]])                      
 >> array([[ 0.00...,  0.99...]])
```
具体的惩罚方法可以通过 penalty 参数来设定。 SGD 支持以下 penalties（惩罚）:

- penalty="l2": L2 norm penalty on coef_,
- penalty="l1": L1 norm penalty on coef_,
- penalty="elasticnet": Convex combination of L2 and L1（L2 型和 L1 型的凸组合）; (1 - l1_ratio) * L2 + l1_ratio * L1.

默认设置为 penalty="l2" 。 L1 penalty （惩罚）导致稀疏解，使得大多数系数为零。 Elastic Net（弹性网）解决了在特征高相关时 L1 penalty（惩罚）的一些不足。参数 l1_ratio 控制了 L1 和 L2 penalty（惩罚）的 convex combination （凸组合）。

### 回归

SGDRegressor 类实现了一个简单的随机梯度下降学习例程，它支持用不同的损失函数和惩罚来拟合线性回归模型。 SGDRegressor 非常适用于有大量训练样本（>10.000)的回归问题，对于其他问题，我们推荐使用 Ridge ，Lasso ，或 ElasticNet 。

具体的损失函数可以通过 loss 参数设置。 SGDRegressor 支持以下的损失函数:

- loss="squared_loss": Ordinary least squares（普通最小二乘法）,
- loss="huber": Huber loss for robust regression（Huber回归）,
- loss="epsilon_insensitive": linear Support Vector Regression（线性支持向量回归）.

Huber 和 epsilon-insensitive 损失函数可用于 robust regression（鲁棒回归）。不敏感区域的宽度必须通过参数 epsilon 来设定。这个参数取决于目标变量的规模。SGDRegressor 支持 ASGD（平均随机梯度下降） 作为 SGDClassifier。 均值化可以通过设置 `average=True` 来启用。对于利用了 squared loss（平方损失）和 l2 penalty（l2惩罚）的回归，在 Ridge 中提供了另一个采取 averaging strategy（平均策略）的 SGD 变体，其使用了随机平均梯度 (SAG) 算法。

class sklearn.linear_model.SGDClassifier(loss=’hinge’, penalty=’l2’, alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=None, tol=None, shuffle=True, verbose=0, epsilon=0.1, n_jobs=1, random_state=None, learning_rate=’optimal’, eta0=0.0, power_t=0.5, class_weight=None, warm_start=False, average=False, n_iter=None)

### 参数

```
    loss : str, default: ‘hinge’
    要是用的损失函数。默认为‘hinge’，这是一个线性支持向量机。
    选项是：‘hinge’, ‘log’, ‘modified_huber’, ‘squared_hinge’, ‘perceptron’, or a regression loss: ‘squared_loss’, ‘huber’, ‘epsilon_insensitive’, or ‘squared_epsilon_insensitive’.
    ‘log’ 损失使逻辑回归成为一种概率分类器。‘modified_huber’是另一个平滑的损失，它对异常值有容错，对概率估计值相同。‘squared_hinge’就像‘hinge’一样，但它的惩罚是平方的。感知器是感知器算法使用的线性损失。其他损失是为回归设计的，但也可以用于分类;参见SGDRegressor获得描述。

    penalty : str, ‘none’, ‘l2’, ‘l1’, or ‘elasticnet’
    使用的惩罚（正则项）。默认值为l2，这是线性SVM模型的标准调节器。‘l1’和‘elasticnet’可能会给模型带来稀疏性(特征选择)，而“l2”则无法做到这一点。

    alpha : float
    乘以正则项的常数。默认值为0.0001，当设置为‘optimal’时，也用于计算learning_rate。
    
    shuffle : bool, optional.
    训练数据是否在一个epoch后重新打乱。默认为是的。（epoch为一个周期，期间遍历所有数据）
    
    random_state :  int, RandomState instance or None, optional (default=None)
    在对数据进行洗牌时，伪随机数生成器的种子。
    - 如果是int，random_state是随机数生成器所使用的种子
    - 如果是RandomState instance，random_state是随机数生成器
    - 如果是None，随机数生成器是np.random所使用的随机状态实例
    默认为None。
```   

### 属性

```
    coef_ : array, shape (1, n_features) if n_classes == 2 else (n_classes, n_features)
    分配给特征的权重.
    
    intercept_ : array, shape (1,) if n_classes == 2 else (n_classes,)
    决策函数中的常量.
    
    n_iter_ : int
    达到停止标准的实际迭代次数。对于多类拟合，它是所有二进制拟合的最大值. 

    loss_function_ : 具体的损失函数.
```

### 方法

```
    decision_function(X)	                           预测样本的置信度。
    densify()	                                      将系数矩阵转换成密集数组格式。
    fit(X, y[, coef_init, intercept_init, …])	     拟合适用于随机梯度下降的线性模型。
    get_params([deep])	                             获取参数。
    partial_fit(X, y[, classes, sample_weight])        拟合具有随机梯度下降的线性模型。
    predict(X)	                                     为X中的样本预测类标签。
    score(X, y[, sample_weight])	                   返回给定测试数据和标签的平均精度。
    sparsify()	                                     转换系数矩阵为稀疏格式。
```

## 3.2.2Perceptron

感知器（perceptron）是二类分类的线性分类模型，其输入为实例的特征向量，输出为实例的类别。感知机对应于输入空间（特征空间）中将实例划分为正负两类的分离超平面。感知机是一种线性分类模型。

Perceptron 是适用于大规模学习的一种简单算法。默认情况下：

- 不需要设置学习率（learning rate），
- 不需要正则化处理，
- 仅使用错误样本更新模型。


```
>>class sklearn.linear_model.Perceptron(penalty=None, alpha=0.0001, fit_intercept=True, max_iter=None, tol=None, shuffle=True, verbose=0, eta0=1.0, n_jobs=1, random_state=0, class_weight=None, warm_start=False, n_iter=None)
```

### 参数

    penalty（惩罚） : None, ‘l2’ or ‘l1’ or ‘elasticnet’
    使用的惩罚（正则项），默认为没有。
    
    alpha : float
    如果使用正则化，就会使正则项成倍增加。默认为0.0001。
    
    shuffle : bool, optional, default True
    训练数据是否在一个epoch后重新打乱。默认为是的。（epoch为一个周期，期间遍历所有数据）
    
    random_state : int, RandomState instance or None, optional, default None
    在对数据进行洗牌时，伪随机数生成器的种子。
    - 如果是int，random_state是随机数生成器所使用的种子
    - 如果是RandomState instance，random_state是随机数生成器
    - 如果是None，随机数生成器是np.random所使用的随机状态实例
    默认为None。

### 属性

    coef_ : array, shape = [1, n_features] if n_classes == 2 else [n_classes, n_features]
    分配给特征的权重。
    
    intercept_ : array, shape = [1] if n_classes == 2 else [n_classes]
    决策函数中的常量。
    
    n_iter_ : int
    达到停止标准的实际迭代次数。对于多类拟合，它是所有二进制拟合的最大值。

### 方法

```
    decision_function(X)                           预测样本的置信度。
    fit(X, y[, coef_init, intercept_init, …])     拟合适用于随机梯度下降的线性模型。
    predict(X)                                     为X中的样本预测类标签。    
    score(X, y[, sample_weight])                   返回给定测试数据和标签的平均精度。    
    get_params([deep])                             获取这个估计量的参数。
```